In [ ]:
import sys
sys.path.insert(1, '../Lib/')
import numpy as np
import pickle, os, lib, rep_lib, json, datetime
import networkx as nx

## Create Dataset for Repetita

### Make sure to point the path to the folder containing the Repetita files

In [ ]:
path_repetita = "../Dataset/Repetita"
path_TM= "../TM"
path_results = "../Results"

In [ ]:
def create_dataset(topology):
    for scale in [0.1, 0.5, 0.75, 1]:
        for number in range(5):
            traffic = rep_lib.load_traffic(f"{path_repetita}/data/{topology}.000{number}.demands")


            t = np.empty((len(traffic), 1),dtype=tuple)
            for i, element in enumerate(traffic):
                traffic[i] = (element[0], element[1]*scale)
                t[i,0] = traffic[i]

            G = rep_lib.load_topo(f"{path_repetita}/data/{topology}.graph", traffic)
            if not os.path.exists(f"{path_TM}/Repetita/{topology}/"):
                os.makedirs(f"{path_TM}/Repetita/{topology}/")
            file = open(f"{path_TM}/Repetita/{topology}/000{number}_{scale}.pkl", 'wb')
            pickle.dump((G, t), file)
    print(len(G.edges)/2)
    print(len(G.edges)/ (2* len(G.nodes)))
    util, total_bw = rep_lib.get_utilization(G)
    print(util/total_bw)

path = f"{path_repetita}/data/2016TopologyZooUCL_inverseCapacity/"
for name in os.listdir(path):
    folder = path.split("/")[-2]
    if name.endswith(".graph"):
        print(name)
        topo_name = name.split(".")[0]
        print(f"{folder}/{topo_name}")
        create_dataset(f"{folder}/{topo_name}")

path = f"{path_repetita}/data/2016RocketFuelUCL/"
for name in os.listdir(path):
    folder = path.split("/")[-2]
    if name.endswith(".graph"):
        print(name)
        topo_name = name.split(".")[0]
        print(f"{folder}/{topo_name}")
        create_dataset(f"{folder}/{topo_name}")


In [ ]:
def eval(topo):
    result = []
    scales = [0.1,0.5,0.75,1]
    timesteps = [0,1,2,3,4]

    for scale in scales:

        for i in timesteps:
            
            ts = i
            links_off = []
            file = open(f"{path_TM}/Repetita/{topo}/000{ts}_{scale}.pkl", "rb")
            G, t = pickle.load(file)
            budget = lib.get_reroute_budget(G)
            result.append(dict(config={"scale":scale,"timestep":ts,"reroute_budget": budget, "topo": topo}))
            topo_graph = nx.MultiGraph()
            topo_graph.add_nodes_from(G.nodes())
            sleep_edges = []

            for edge in G.edges:
                if topo_graph.has_edge(edge[1],edge[0],tuple(reversed(edge[2]))):
                    continue
                else:
                    topo_graph.add_edge(edge[0], edge[1], key=edge[2], sleep=False, max_bw=G[edge[0]][edge[1]][edge[2]]["max_bw"])
                    sleep_edges.append(edge)

            edges_to_sleep = lib.get_links_to_sleep(sleep_edges, G, link_margin=0.2)
            edges_to_sleep = lib.optimize_link_sleep(edges_to_sleep, G)
            
            for sleep_edge, score in edges_to_sleep:
                if budget - score < 0:
                    break
                if lib.check_connectedness(sleep_edge, topo_graph) == False:
                    continue

                budget -= score
                topo_graph[sleep_edge[0]][sleep_edge[1]][sleep_edge[2]]["sleep"] = True
                links_off.append(sleep_edge)
            result[-1]["sleep_edges"] = links_off
            result[-1]["ol_now"] = lib.check_overload(f"{path_TM}/Repetita/{topo}/000{ts}_{scale}.pkl", topo_graph)

    for element in result:
        ol_now = (len(element["ol_now"]["ol_sleep"])-len(element["ol_now"]["ol_active"]), len(element["ol_now"]["ol_sleep_1"])-len(element["ol_now"]["ol_active_1"]))
        element["result"] = {"scale": element["config"]["scale"] ,"ol_now":ol_now, "no_sleep": len(element["sleep_edges"])}
        print(element["result"])

    if not os.path.exists(f"{path_results}/repetita/"):
            os.makedirs(f"{path_results}/repetita/")
    lib.save_result(f"{path_results}/repetita/", result)
    return

path = f"{path_TM}/Repetita/2016TopologyZooUCL_inverseCapacity/"
for name in os.listdir(path):
    folder = path.split("/")[-2]
    topo = f"{folder}/{name}"
    print(topo)
    eval(topo)

path = f"{path_TM}/Repetita/2016RocketFuelUCL/"
for name in os.listdir(path):
    folder = path.split("/")[-2]
    topo = f"{folder}/{name}"
    print(topo)
    eval(topo)

## Show results of Repetita experiment

In [ ]:
def show_repetita(path, show=False):
    dataset = {}

    for i in [0.1,0.5,0.75,1]:
        with open(f"{path}/scale_{i}.txt", "r") as f:
            dataset[f"scale_{i}"] = []
            lines = f.readlines()
            for line in lines:
                line = line.split(",")
                dataset[f"scale_{i}"].append((line[0], json.loads(','.join(line[1:]))))

    for key in dataset.keys():

        result_dict = {"no_sleep": [], "ol": [], "ol_1": [], "ts": [], "best_case": [], "no_of_links": [], "no_of_nodes": [], "avg_degree":[],"avg_util":[], "diameter":[], "actual_sleep": []}
        for i, element in enumerate(dataset[key]):
            topo = element[1]["config"]["topo"]
            ts = element[1]["config"]["timestep"]
            scale = element[1]["config"]["scale"]
            file = open(f"{path_TM}/Repetita/{topo}/000{ts}_{scale}.pkl", "rb")
            G, t = pickle.load(file)
            result_dict["best_case"].append(len(G.edges)/2 - len(G.nodes) + 1)
            result_dict["no_of_nodes"].append(len(G.nodes))
            result_dict["no_of_links"].append(len(G.edges)/2)
            utilizations = []

            for edge in G.edges:
                utilizations.append((G[edge[0]][edge[1]][edge[2]]["usage"],G[edge[0]][edge[1]][edge[2]]["max_bw"]))

            utilizations = np.array(utilizations)
            abs_util = np.sum(utilizations[:,0])
            util = np.sum(utilizations[:,0])/np.sum(utilizations[:,1])
            result_dict["avg_util"].append(util)
            H = nx.MultiGraph(G)
            result_dict["avg_degree"].append(np.mean([x[1] for x in H.degree]))
            result_dict["diameter"].append(nx.diameter(G))
            ol_now = element[1]["result"]["ol_now"]
            result_dict["ts"].append(datetime.datetime.fromtimestamp((element[1]["config"]["timestep"])))
            result_dict["no_sleep"].append(element[1]["result"]["no_sleep"])
            result_dict["ol"].append(ol_now[0])
            result_dict["ol_1"].append(ol_now[1])
            if ol_now == [0,0]:
                result_dict["actual_sleep"].append(element[1]["result"]["no_sleep"])
            else:
                result_dict["actual_sleep"].append(0)

        if show:
            print(key)
            print(f'best_case: {np.mean(result_dict["best_case"])}, no_of_links: {np.mean(result_dict["no_of_links"])}, no_of_nodes: {np.mean(result_dict["no_of_nodes"])}')
            print(f'avg_degree: {np.mean(result_dict["avg_degree"])}, avg_util: {np.mean(result_dict["avg_util"])} , diameter: {np.mean(result_dict["diameter"])}')
            print(f'no of timesteps: {len(result_dict["no_sleep"])}')
            print(f'links turned off: {np.mean(result_dict["no_sleep"])}, actual after correction: {np.mean(result_dict["actual_sleep"])}')
            print(f'Avg. link over 80%: {np.mean(result_dict["ol"])}, no. of times it happens: {np.count_nonzero(result_dict["ol"])}')
            print(f'Avg. link over 100%: {np.mean(result_dict["ol_1"])}, no. of times it happens: {np.count_nonzero(result_dict["ol_1"])}')

            print()
            

In [ ]:
path = f"{path_results}/repetita/"
show_repetita(path, show=True)